In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from utils import load_data, get_similar_players_cosine, get_similar_players_knn, compare_players

df, scaled_df, similarity_df = load_data()

2023-12-17 14:07:46.428 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-17 14:07:46.541 
  command:

    streamlit run /home/asus/anaconda3/envs/datamining/lib/python3.8/site-packages/ipykernel_launcher.py [ARGUMENTS]
2023-12-17 14:07:46.542 No runtime found, using MemoryCacheStorageManager
/home/asus/stuDYING/IT/DataScience/project/get_data/get_data.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM all_leagues',conn)


In [4]:
from features_meaning import dict_metric

type_of_stats = {'Passing': ['Cmp', 'PassAtt', 'Cmp%', 'PassTotDist', 'PassPrgDist', 'Cmp.1', 'Att.1', 'Cmp%.1', 'Cmp.2', 'Att.2', 'Cmp%.2', 'Cmp.3', 'Att.3', 'Cmp%.3', 'Ast', 'xA', 'KP', 'PassFinThird', 'PPA', 'CrsPA', 'PassProg'],
                 'Goal and Shot Creation': ['SCA', 'PassLiveShot', 'PassDeadShot', 'DribShot', 'ShLSh', 'Fld', 'DefShot', 'GCA', 'PassLiveGoal', 'PassDeadGoal', 'DribGoal', 'ShGoal', 'FldGoal', 'DefGoal', 'OG'],
                 'Defensive Actions': ['Tkl', 'TklW', 'TacklesDef3rd', 'TacklesMid3rd', 'TacklesAtt3rd', 'DribTackled', 'DribContest', 'DribTackled%', 'Past', 'Press', 'SuccPress', 'SuccPress%', 'PressDef3rd', 'PressMid3rd', 'PressAtt3rd', 'Blocks', 'BlockSh', 'ShSv', 'Pass', 'Int', 'Tkl+Int', 'Clr', 'Err'],
                 'Possession': ['Touches', 'Def Pen', 'TouchDef3rd', 'TouchMid3rd', 'TouchAtt3rd', 'AttPen', 'Live', 'Succ', 'Att', 'Succ%', '#Pl', 'Megs', 'Carries', 'TotDist', 'PrgDist', 'ProgCarries', 'CarriesFinThird', 'CPA', 'Mis', 'Dis', 'Targ', 'Rec', 'Rec%'],
                 'Summary': ['ProgPassRec', 'Gls', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY', 'CrdR', 'xG', 'npxG']}

In [5]:
player_name = 'Robert Lewandowski'

player_row = df[df['Name'] == player_name]
player_position = player_row['Pos'].values[0][0]
same_position_players = df[df['Pos'].apply(lambda x: player_position in x)]

In [6]:
same_position_players

,player_id,Name,Squad,Min,Pos,#Pl,Ast,Att,Att.1,Att.2,Att.3,AttPen,BlockSh,Blocks,CPA,Carries,CarriesFinThird,Clr,Cmp,Cmp.1,Cmp.2,Cmp.3,CrdR,CrdY,CrsPA,Def Pen,DefGoal,DefShot,Dis,DribContest,DribGoal,DribShot,DribTackled,Err,Fld,FldGoal,GCA,Gls,Int,KP,Live,Megs,Mis,PK,PKatt,PPA,Pass,PassAtt,PassDeadGoal,PassDeadShot,PassFinThird,PassLiveGoal,PassLiveShot,PassPrgDist,PassProg,PassTotDist,Past,Press,PressAtt3rd,PressDef3rd,PressMid3rd,PrgDist,ProgCarries,ProgPassRec,Rec,SCA,Sh,ShGoal,ShLSh,ShSv,SoT,Succ,SuccPress,TacklesAtt3rd,TacklesDef3rd,TacklesMid3rd,Targ,Tkl,Tkl+Int,TklW,TotDist,TouchAtt3rd,TouchDef3rd,TouchMid3rd,Touches,npxG,xA,xG,Cmp%,Cmp%.1,Cmp%.2,Cmp%.3,DribTackled%,SuccPress%,Succ%,Rec%
0,e6adb385a0d7c823cf886437eeb23046,Aaron Connolly,Brighton,791.0,"[FW, LW, WB]",0.0,0.113780,1.137800,0.0,0.0,0.0,4.323641,0.113780,1.024020,0.455120,14.108723,0.910240,0.113780,8.988622,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.706700,0.568900,0.000000,0.341340,0.113780,0.0,0.227560,0.000000,0.113780,0.227560,0.000000,0.568900,21.731985,0.000000,3.299621,0.000000,0.000000,0.113780,0.910240,11.491783,0.000000,0.000000,0.227560,0.113780,0.796460,18.773704,0.341340,130.505689,0.455120,14.108723,6.713021,0.796460,6.599241,44.032870,1.934260,7.509482,16.270544,1.365360,2.616941,0.000000,0.000000,0.0,0.910240,0.910240,4.551201,0.113780,0.227560,0.455120,40.619469,0.796460,0.0,0.568900,74.525917,13.198483,1.592920,9.557522,22.869785,0.409608,0.011378,0.409608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ecb6284980af030d92a131f88c2ec121,Aaron Hickey,Bologna,758.0,"[LB, WB, FW]",0.0,0.000000,1.781003,0.0,0.0,0.0,1.187335,0.237467,2.018470,0.237467,38.232190,2.137203,1.781003,40.250660,0.0,0.0,0.0,0.118734,0.356201,0.118734,0.0,0.000000,0.000000,1.543536,2.018470,0.000000,0.000000,0.712401,0.0,0.000000,0.000000,0.118734,0.000000,1.424802,0.474934,50.699208,0.000000,1.187335,0.000000,0.000000,0.474934,1.781003,47.968338,0.000000,0.118734,2.849604,0.118734,0.712401,234.736148,2.849604,724.274406,1.306069,11.160950,1.781003,5.343008,4.036939,113.034301,4.036939,1.899736,33.245383,0.831135,0.356201,0.000000,0.000000,0.0,0.118734,0.949868,3.918206,0.118734,1.187335,0.237467,35.738786,1.543536,0.0,0.712401,229.274406,16.503958,18.997361,29.327177,59.960422,0.011873,0.035620,0.011873,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,c3a9b84dd52f619a84b7b5c6f8bd464c,Aaron Leya Iseka,Metz,1008.0,"[FW, AM]",0.0,0.000000,2.053571,0.0,0.0,0.0,3.125000,0.267857,0.535714,0.892857,18.303571,1.071429,0.357143,13.482143,0.0,0.0,0.0,0.000000,0.357143,0.000000,0.0,0.000000,0.000000,1.785714,0.892857,0.089286,0.267857,0.267857,0.0,0.357143,0.000000,0.089286,0.357143,0.178571,1.071429,27.500000,0.000000,2.857143,0.178571,0.267857,0.267857,0.267857,17.767857,0.000000,0.000000,0.892857,0.000000,1.250000,30.357143,0.803571,202.589286,0.625000,10.803571,5.446429,0.357143,5.000000,38.392857,2.678571,5.535714,22.500000,1.875000,1.607143,0.000000,0.000000,0.0,0.535714,1.250000,2.321429,0.267857,0.267857,0.267857,48.482143,0.803571,0.0,0.446429,75.803571,11.517857,2.946429,16.071429,28.482143,0.214286,0.062500,0.428571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,60f3a791358df47f263079e53eb762e4,Aaron Martin,Celta Vigo,1540.0,"[LB, WB, FW]",0.0,0.175325,1.402597,0.0,0.0,0.0,0.993506,0.233766,1.519481,0.233766,36.876623,1.577922,2.922078,40.383117,0.0,0.0,0.0,0.000000,0.058442,0.233766,0.0,0.000000,0.058442,0.642857,2.103896,0.000000,0.000000,0.759740,0.0,0.058442,0.000000,0.175325,0.000000,0.993506,0.409091,50.785714,0.000000,0.350649,0.000000,0.000000,0.525974,1.285714,50.376623,0.000000,0.000000,1.811688,0.175325,0.584416,220.967532,2.629870,689.318182,1.344156,11.629870,1.285714,4.909091,5.435065,81.876623,4.733766,2.396104,36.000000,0.759740,0.350649,0.000000,0.058442,0.0,0.058442,0.876623,2.922078,0.116883,1.051948,0.876623,40.090909,2.045455,0.0,0.701299,153.993506,15.370130,18.116883,29.629870,60.136364,0.017532,0.070130,0.017532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,835c5f0e942b1c5ca

In [7]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from features_meaning import dict_metric

In [8]:
df_temp = same_position_players.copy()

position = df[df['Name'] == player_name]['Pos'].values[0][0]
lst_cols_pr = [col for col in df.columns if df[col].dtype == 'float64' and col != 'Min']

for col in lst_cols_pr:
    df_temp[col] = df_temp[col].rank(pct=True)

df_player_pr = df_temp[(df_temp['Name'] == player_name)]

df_player_pr = df_player_pr[lst_cols_pr]

df_player_pr_t = df_player_pr.T

df_player_pr_t = df_player_pr_t.reset_index(drop=False)

df_player_pr_t.columns = ['Stats', 'PR']

dict_metrics = {k: dict_metric[k] for k in lst_cols_pr}

df_player_pr_t['Metric'] = df_player_pr_t['Stats'].map(dict_metrics)

df_player_pr_t

,Stats,PR,Metric
0,#Pl,0.500947,Number of Players Dribbled Past
1,Ast,0.867424,Assists
2,Att,0.395833,Dribbles Attempted
3,Att.1,0.500947,Passes Attempted Short
4,Att.2,0.500947,Passes Attempted Medium
...,...,...,...
86,Cmp%.3,0.500947,Pass Completion % Long
87,DribTackled%,0.500947,Percentage of times dribble tackled
88,SuccPress%,0.500947,Percentage of successful pressures
89,Succ%,0.500947,Dribble Success %


In [ ]:
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go

# Divide the stats into 5 major types
passing_stats = type_of_stats['Passing']
goal_stats = type_of_stats['Goal and Shot Creation']
defensive_stats = type_of_stats['Defensive Actions']
possession_stats = type_of_stats['Possession']
summary_stats = type_of_stats['Summary']

# Filter the DataFrame for each major type
passing_df = df_player_pr_t[df_player_pr_t['Stats'].isin(passing_stats)]
goal_df = df_player_pr_t[df_player_pr_t['Stats'].isin(goal_stats)]
defensive_df = df_player_pr_t[df_player_pr_t['Stats'].isin(defensive_stats)]
possession_df = df_player_pr_t[df_player_pr_t['Stats'].isin(possession_stats)]
summary_df = df_player_pr_t[df_player_pr_t['Stats'].isin(summary_stats)]

# Calculate the maximum number of features among all subplots
max_features = max(len(summary_stats), len(passing_stats), len(goal_stats), len(defensive_stats), len(possession_stats))

# Calculate the width of the bars based on the maximum number of features
bar_width = 0.04

row_heights=[0.5, 0.8, 0.8, 0.7, 0.8]

# Add the subplots to the big plot with updated row_heights
fig = sp.make_subplots(rows=5, cols=1, shared_xaxes=True, vertical_spacing=0.02, subplot_titles=('Summary Stats', 'Passing Stats', 'Goal and Shot Creation Stats', 'Defensive Action Stats', 'Possession Stats'))

# Rest of the code...
# Add the subplots to the big plot
fig.add_trace(go.Bar(x=summary_df['PR'], y=summary_df['Metric'], orientation='h',
                    width=bar_width*len(summary_stats),
                    marker=dict(color=summary_df['PR'], coloraxis='coloraxis')), row=1, col=1)
fig.add_trace(go.Bar(x=passing_df['PR'], y=passing_df['Metric'], orientation='h',
                     width=bar_width*len(passing_stats),
                     marker=dict(color=passing_df['PR'], coloraxis='coloraxis')), row=2, col=1)
fig.add_trace(go.Bar(x=goal_df['PR'], y=goal_df['Metric'], orientation='h',
                     width=bar_width*len(goal_stats),
                     marker=dict(color=goal_df['PR'], coloraxis='coloraxis')), row=3, col=1)
fig.add_trace(go.Bar(x=defensive_df['PR'], y=defensive_df['Metric'], orientation='h',
                     width=bar_width*len(defensive_stats),
                     marker=dict(color=defensive_df['PR'], coloraxis='coloraxis')), row=4, col=1)
fig.add_trace(go.Bar(x=possession_df['PR'], y=possession_df['Metric'], orientation='h',
                     width=bar_width*len(possession_stats),
                     marker=dict(color=possession_df['PR'], coloraxis='coloraxis')), row=5, col=1)

# Update the layout of the big plot
fig.update_layout(
    height=4000,
    width=1600,
    coloraxis=dict(colorscale='RdYlGn', colorbar=dict(title='Percentile Rank')),
    bargap=0.05,
    bargroupgap=0.05,
    font=dict(
        family="Arial",
        size=18,
        color="black"
    )
)

for trace in fig['data']: 
    trace['showlegend'] = False

fig.update_yaxes(automargin='left+right')

# Show the big plot
fig.show()

In [30]:
def plot_percentiles(df, player_name='Harry Kane'):

    player_row = df[df['Name'] == player_name]
    player_position = player_row['Pos'].values[0][0]
    same_position_players = df[df['Pos'].apply(lambda x: player_position in x)]

    df_temp = same_position_players.copy()

    # position = df[df['Name'] == player_name]['Pos'].values[0][0]
    lst_cols_pr = [col for col in df.columns if df[col].dtype == 'float64' and col != 'Min']

    for col in lst_cols_pr:
        df_temp[col] = df_temp[col].rank(pct=True)

    df_player_pr = df_temp[(df_temp['Name'] == player_name)]

    df_player_pr = df_player_pr[lst_cols_pr]

    df_player_pr_t = df_player_pr.T

    df_player_pr_t = df_player_pr_t.reset_index(drop=False)

    df_player_pr_t.columns = ['Stats', 'PR']

    dict_metrics = {k: dict_metric[k] for k in lst_cols_pr}

    df_player_pr_t['Metric'] = df_player_pr_t['Stats'].map(dict_metrics)

    df_player_pr_t

    passing_stats = type_of_stats['Passing']
    goal_stats = type_of_stats['Goal and Shot Creation']
    defensive_stats = type_of_stats['Defensive Actions']
    possession_stats = type_of_stats['Possession']
    summary_stats = type_of_stats['Summary']

    # Filter the DataFrame for each major type
    passing_df = df_player_pr_t[df_player_pr_t['Stats'].isin(passing_stats)]
    goal_df = df_player_pr_t[df_player_pr_t['Stats'].isin(goal_stats)]
    defensive_df = df_player_pr_t[df_player_pr_t['Stats'].isin(defensive_stats)]
    possession_df = df_player_pr_t[df_player_pr_t['Stats'].isin(possession_stats)]
    summary_df = df_player_pr_t[df_player_pr_t['Stats'].isin(summary_stats)]

    # Calculate the maximum number of features among all subplots
    # max_features = max(len(summary_stats), len(passing_stats), len(goal_stats), len(defensive_stats), len(possession_stats))

    # Calculate the width of the bars based on the maximum number of features
    bar_width = 0.04

    # row_heights=[0.5, 0.8, 0.8, 0.7, 0.8]

    # Add the subplots to the big plot with updated row_heights
    fig = sp.make_subplots(rows=5, cols=1, shared_xaxes=True, vertical_spacing=0.02, subplot_titles=('Summary Stats', 'Passing Stats', 'Goal and Shot Creation Stats', 'Defensive Action Stats', 'Possession Stats'))

    # Add the subplots to the big plot
    fig.add_trace(go.Bar(x=summary_df['PR'], y=summary_df['Metric'], orientation='h',
                        width=bar_width*len(summary_stats),
                        marker=dict(color=summary_df['PR'], coloraxis='coloraxis')), row=1, col=1)
    fig.add_trace(go.Bar(x=passing_df['PR'], y=passing_df['Metric'], orientation='h',
                        width=bar_width*len(passing_stats),
                        marker=dict(color=passing_df['PR'], coloraxis='coloraxis')), row=2, col=1)
    fig.add_trace(go.Bar(x=goal_df['PR'], y=goal_df['Metric'], orientation='h',
                        width=bar_width*len(goal_stats),
                        marker=dict(color=goal_df['PR'], coloraxis='coloraxis')), row=3, col=1)
    fig.add_trace(go.Bar(x=defensive_df['PR'], y=defensive_df['Metric'], orientation='h',
                        width=bar_width*len(defensive_stats),
                        marker=dict(color=defensive_df['PR'], coloraxis='coloraxis')), row=4, col=1)
    fig.add_trace(go.Bar(x=possession_df['PR'], y=possession_df['Metric'], orientation='h',
                        width=bar_width*len(possession_stats),
                        marker=dict(color=possession_df['PR'], coloraxis='coloraxis')), row=5, col=1)

    # Update the layout of the big plot
    fig.update_layout(
        # height=3000,
        # width=1000,
        coloraxis=dict(colorscale='RdYlGn', colorbar=dict(title='Percentile Rank')),
        bargap=0.05,
        bargroupgap=0.05,
        font=dict(
            family="Arial",
            size=14,
            color="black"
        )
    )

    for trace in fig['data']: 
        trace['showlegend'] = False

    fig.update_yaxes(automargin='left+right')

    return fig

plot_percentiles(df, player_name='Robert Lewandowski')